In [1]:
import gym
from IPython.display import clear_output
import numpy as np
import time
from pprint import pprint

In [2]:
env = gym.make('Taxi-v3')

In [3]:
gamma = 0.8

In [4]:
# Define State Action Space
state_space_n = env.observation_space.n
action_space_n = env.action_space.n
state_action_space_n = state_space_n * action_space_n
# Map State-Action
def msa(state, action):
    return state * action_space_n + action
# Build Reward & Transition Matrix
R = np.zeros([state_action_space_n])
P = np.zeros([state_action_space_n, state_space_n])
for state in range(state_space_n):
    for action in range(action_space_n):
        for p, s, r, _ in env.P[state][action]:
            R[msa(state, action)] += p * r
            P[msa(state, action), s] = p

In [5]:
%time
# Calculate Optimal Q Values
Q = np.zeros([state_action_space_n])
PI = np.zeros([state_space_n, state_action_space_n])
I = np.identity(state_action_space_n)
for i in range(100):
    print(i)
    # Build Policy Matrix
    old_PI = PI.copy()
    PI.fill(0)
    for state in range(state_space_n):
        PI[state, msa(state,0) + np.argmax(Q[msa(state,0):msa(state,action_space_n)])] = 1
    print('PI', np.array_equal(old_PI, PI), state_space_n - sum(np.sum(old_PI == PI, axis=1) == state_action_space_n))
    # Calculate Q Values
    old_Q = Q
    # Q = np.linalg.solve(I - gamma * P @ PI, R)
    Q = np.linalg.inv(I - gamma * P @ PI) @ R
    print('Q ', np.array_equal(Q, old_Q), state_action_space_n - sum(Q == old_Q))
    if np.array_equal(Q, old_Q): break

Wall time: 0 ns
0
PI False 500
Q  False 3000
1
PI False 4
Q  False 72
2
PI False 20
Q  False 240
3
PI False 32
Q  False 261
4
PI False 34
Q  False 247
5
PI False 39
Q  False 228
6
PI False 37
Q  False 276
7
PI False 41
Q  False 327
8
PI False 57
Q  False 356
9
PI False 49
Q  False 325
10
PI False 51
Q  False 323
11
PI False 41
Q  False 241
12
PI False 33
Q  False 201
13
PI False 23
Q  False 134
14
PI False 20
Q  False 112
15
PI False 12
Q  False 47
16
PI False 3
Q  False 18
17
PI True 0
Q  True 0


In [6]:
# state = env.reset()
# while True:
#     action = np.argmax(Q[msa(state,0):msa(state,action_space_n)])
#     state, reward, done, info = env.step(action)

#     clear_output(wait=True)
#     env.render()
#     time.sleep(0.25)
#     if done: break
# env.close()

In [7]:
# %%time
# # Calculate Optimal V Values
# V = np.zeros([state_space_n])
# PI = np.zeros([state_space_n, state_action_space_n])
# I = np.identity(state_space_n)
# i = 0
# while True:
#     print(i)
#     i += 1
#     # Build Policy Matrix
#     old_PI = PI.copy()
#     PI.fill(0)
#     for state in range(state_space_n):
#         max_action = -1
#         max_value = float('-inf')
#         for action in range(action_space_n):
#             value = sum([p * (r + V[s]) for p, s, r, _ in env.P[state][action]])
#             if value > max_value:
#                 max_action = action
#                 max_value = value
#         PI[state, msa(state, max_action)] = 1
#     print('PI', np.array_equal(old_PI, PI), state_space_n - sum(np.sum(old_PI == PI, axis=1) == state_action_space_n))
#     # Calculate V Values
#     old_V = V
#     V = np.linalg.solve(I - gamma * PI @ P - 1, PI @ R)
#     # V = np.linalg.pinv(I - gamma * PI @ P) @ (PI @ R)
#     print('V ', np.array_equal(V, old_V), state_space_n - sum(V == old_V))
#     if np.array_equal(V, old_V): break